In [1]:
%cd ../..

/home/mjozwiak/uni/mgr


In [3]:
import tensorflow as tf
tf.enable_eager_execution()

from graph_neurawkes.experiments import calculate_everything
from graph_neurawkes.data.edgelist_data import utils as edge_utils
from graph_neurawkes.src.models import Neurawkes, GraphNeurawkes

## Processing utils

In [3]:
def df_to_list(df):
    return [(int(s), int(r), t) for s,r,t in df.values.tolist()]

## Constants

In [4]:
VAL_RATIO = 0.2
PRED_RATIO = 0.2

#### Fb-forum

In [ ]:
DATA_PATH_FB = 'data/edgelist_data/fb-forum/data.csv'
GAP_SIZE_FB = 5000
NUM_VERTICES_FB = 899

In [5]:
tf.reset_default_graph()
SAVE_PATH_FB = 'saves/gnh_fb-forum/1'
GNH_MODEL_FB = GraphNeurawkes(64, NUM_VERTICES_FB, 50, True)

#### Radoslaw

In [ ]:
DATA_PATH_RAD = 'data/edgelist_data/ia-radoslaw-email/data.csv'
GAP_SIZE_RAD = 45000
NUM_VERTICES_RAD = 167

In [5]:
tf.reset_default_graph()
SAVE_PATH_RAD = 'saves/gnh_radoslaw/25'
GNH_MODEL_RAD = GraphNeurawkes(64, NUM_VERTICES_RAD, 50, True)

#### Hypertext

In [ ]:
DATA_PATH_HYP = 'data/edgelist_data/ia-contacts_hypertext2009/data.csv'
GAP_SIZE_HYP = 75
NUM_VERTICES_HYP = 113

In [5]:
tf.reset_default_graph()
SAVE_PATH_HYP = 'saves/gnh_hypertext/13'
GNH_MODEL_HYP = GraphNeurawkes(64, NUM_VERTICES_FB, 50, False)

## Processing functions

In [10]:
def get_longest_val_sequence(data_path, gap_size):
    df = edge_utils._get_df_from_csv(data_path)
    num_ids = max(df.sender.max(), df.recipient.max()) + 1

    dfs = edge_utils.cut_on_big_gaps(df, gap_size)
    val_count = int(len(dfs) * VAL_RATIO)
    val_dfs = dfs[:val_count]
    
    return df_to_list(max(val_dfs, key=lambda d: len(d)))

In [7]:
def calculate_stats_for_seq(seq, mode):
    true_len = int(len(seq) * PRED_RATIO)
    seed_seq, true_seq = seq[:-true_len], seq[-true_len:]

    if mode == 'length':
        pred_seq = GNH_MODEL_FB.generate(saved_path=SAVE_PATH_FB, seed=seed_seq, max_events=len(true_seq))
    elif mode == 'duration':
        pred_seq = GNH_MODEL_FB.generate(saved_path=SAVE_PATH_FB, seed=seed_seq, max_time=true_seq[-1][2])

    return calculate_everything(pred_seq, true_seq, NUM_VERTICES_FB)

## THE MEAT

### Jaccard-esque

In [11]:
val_seq = get_longest_val_sequence(DATA_PATH_FB, GAP_SIZE_FB)
calculate_stats_for_seq(val_seq)

### Synthetic tests